##  1. Откроем файл с данными и изучим общую информацию

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1.1. Импорт библиотек pandas, numpy, datetime, matplotlib.pyplot, seaborn

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
from matplotlib import pyplot as plt
from scipy import stats as st
import seaborn as sb
import math as mth
from plotly import graph_objects as go
pd.set_option('display.max_columns', None) #настройка, чтобы pandas не урезал столбцы

### 1.2. Загрузка данных из CSV-файла, в качестве разделителя знак табуляции \t

In [3]:
marketing_events, events, users, participants = (
    pd.read_csv('/content/drive/MyDrive/Data/final_ab_project_marketing_events.csv'),
    pd.read_csv('/content/drive/MyDrive/Data/final_ab_events.csv'),
    pd.read_csv('/content/drive/MyDrive/Data/final_ab_new_users.csv'),
    pd.read_csv('/content/drive/MyDrive/Data/final_ab_participants.csv')
)

### 1.3. Изучим данные в датафреме marketing_events

In [4]:
display(marketing_events)
marketing_events.info();

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11
5,Black Friday Ads Campaign,"EU, CIS, APAC, N.America",2020-11-26,2020-12-01
6,Chinese New Year Promo,APAC,2020-01-25,2020-02-07
7,Labor day (May 1st) Ads Campaign,"EU, CIS, APAC",2020-05-01,2020-05-03
8,International Women's Day Promo,"EU, CIS, APAC",2020-03-08,2020-03-10
9,Victory Day CIS (May 9th) Event,CIS,2020-05-09,2020-05-11


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes


#### 1.3.1. Преобразуем данные с временем в формат даты в датафреме marketing_events

In [5]:
#преобразуем данные с временем в формат даты
marketing_events['start_dt'] = pd.to_datetime(marketing_events['start_dt'])
marketing_events['finish_dt'] = pd.to_datetime(marketing_events['finish_dt'])

### 1.4. Изучим данные в датафреме events

In [6]:
display(events.head(10))
events.info();

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99
5,831887FE7F2D6CBA,2020-12-07 06:50:29,purchase,4.99
6,6B2F726BFD5F8220,2020-12-07 11:27:42,purchase,4.99
7,BEB37715AACF53B0,2020-12-07 04:26:15,purchase,4.99
8,B5FA27F582227197,2020-12-07 01:46:37,purchase,4.99
9,A92195E3CFB83DBD,2020-12-07 00:32:07,purchase,4.99


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     440317 non-null  object 
 1   event_dt    440317 non-null  object 
 2   event_name  440317 non-null  object 
 3   details     62740 non-null   float64
dtypes: float64(1), object(3)
memory usage: 13.4+ MB


#### 1.4.1. Преобразуем данные с временем в формат даты в датафреме events

In [7]:
events['event_dt'] = pd.to_datetime(events['event_dt'])

#### 1.4.2. Проверка датафрема events на пропуски и дубликаты

In [8]:
print(events.isna().sum())
print(events.duplicated().sum())

user_id            0
event_dt           0
event_name         0
details       377577
dtype: int64
0


In [9]:
#выведем строку с пропуском
print(events[events['details'].isna()].head())

                user_id            event_dt    event_name  details
62740  2E1BF1D4C37EA01F 2020-12-07 09:05:47  product_cart      NaN
62741  50734A22C0C63768 2020-12-07 13:24:03  product_cart      NaN
62742  5EB159DA9DC94DBA 2020-12-07 22:54:02  product_cart      NaN
62743  084A22B980BA8169 2020-12-07 15:25:55  product_cart      NaN
62744  0FC21E6F8FAA8DEC 2020-12-07 06:56:27  product_cart      NaN


Из описания данных можно предположить, что информация в столбце details является дополнительной информацией. Поэтому удалять строки с пропусками не будем

### 1.5. Изучим данные в датафреме users

In [10]:
display(users.head(10))
users.info();

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone
5,137119F5A9E69421,2020-12-07,N.America,iPhone
6,62F0C741CC42D0CC,2020-12-07,APAC,iPhone
7,8942E64218C9A1ED,2020-12-07,EU,PC
8,499AFACF904BBAE3,2020-12-07,N.America,iPhone
9,FFCEA1179C253104,2020-12-07,EU,Android


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     61733 non-null  object
 1   first_date  61733 non-null  object
 2   region      61733 non-null  object
 3   device      61733 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB


#### 1.5.1. Преобразуем данные с временем в формат даты в датафреме users

In [11]:
users['first_date'] = pd.to_datetime(users['first_date'])

#### 1.5.2. Проверка датафрема users на пропуски и дубликаты

In [12]:
print(users.isna().sum())
print(users.duplicated().sum())

user_id       0
first_date    0
region        0
device        0
dtype: int64
0


Пропуски и дубликаты в датафреме users отсутствуют

### 1.6. Изучим данные в датафреме participants

In [13]:
display(participants.head(10))
participants.info();

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test
5,4FF2998A348C484F,A,recommender_system_test
6,7473E0943673C09E,A,recommender_system_test
7,C46FE336D240A054,A,recommender_system_test
8,92CB588012C10D3D,A,recommender_system_test
9,057AB296296C7FC0,B,recommender_system_test


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18268 entries, 0 to 18267
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  18268 non-null  object
 1   group    18268 non-null  object
 2   ab_test  18268 non-null  object
dtypes: object(3)
memory usage: 428.3+ KB


#### 1.6.1. Проверка датафрема participants на пропуски и дубликаты

In [14]:
print(participants.isna().sum())
print(participants.duplicated().sum())

user_id    0
group      0
ab_test    0
dtype: int64
0


Пропуски и дубликаты в датафреме participants отсутствуют

## 2. Подготовка и фильтрация данных согласно техническому заданию

### 2.1. Проверим, есть ли пересечения с конкурирующим тестом в датафреме marketing_events

Проверим, есть ли пересечения в период с 01 декабря 2020 по 31 января 2021 (дата запуска нашего теста 07 декабря 2020, дата завершения 04 января 2021 года).

In [15]:
duble_marketing_events = marketing_events[(marketing_events['start_dt'] > '2020-12-01 00:00:00')&(
    marketing_events['finish_dt']< '2021-01-31 00:00:00')]
print(duble_marketing_events)

                         name        regions   start_dt  finish_dt
0    Christmas&New Year Promo  EU, N.America 2020-12-25 2021-01-03
10  CIS New Year Gift Lottery            CIS 2020-12-30 2021-01-07


Нас интересует регион EU (Европа), поэтому наш тест пересекается только с тестом Christmas&New Year Promo. Видимо этот тест направлен на изучение поведений пользрвателей в период Рождественских праздников. На наш тест это исследование не повлияет, поэтому продолжаем исследование.

### 2.2. Проверим, есть ли пересечения с конкурирующим тестом в датафреме participants

Cогласно ТЗ необходимо изучить тест под названием recommender_system_test

In [16]:
#определим сколько пользователей приняло участие в обоих тестах
participants['ab_test'].value_counts()

interface_eu_test          11567
recommender_system_test     6701
Name: ab_test, dtype: int64

In [17]:
double_users_all = participants.groupby('user_id',as_index=False).agg({'ab_test':'nunique'}).query('ab_test>1')
users_nunique_all = participants['user_id'].nunique()
print('Количество пользователей, одновременно попавших в обе группы:', len(double_users_all))
print('Количество уникальных пользователей, принявших участие в тесте:',users_nunique_all)
print('Процент пользователей, одновременно попавших в обе группы:', (len(double_users_all)/users_nunique_all)*100)

Количество пользователей, одновременно попавших в обе группы: 1602
Количество уникальных пользователей, принявших участие в тесте: 16666
Процент пользователей, одновременно попавших в обе группы: 9.612384495379816


Около 9.6% пользователей попали в оба теста. Посмотрим, что это за пользователи

Расчитала процент пользователей, паповших в оба теста от количества пользователей исследуемого теста. Получилось, что около 24% от пользовтаелей, принявших участие в нашем тесте, также попали в конкурирующий. Это довольно большой показатель, поэтому необходимо рассмотреть этих пользователей подробнее.

In [18]:
print('Процент пользователей, одновременно попавших в обе группы, от пользователей исследуемого теста:',\
      len(double_users_all)/len(participants[participants['ab_test']=='recommender_system_test'])*100)

Процент пользователей, одновременно попавших в обе группы, от пользователей исследуемого теста: 23.906879570213402


In [19]:
double_users_list = double_users_all['user_id']
double_users_list.count()

1602

In [20]:
#создадим датафрем double_users_all_v2 с пользователями,попавшими в обе группы
double_users_all_v2 = participants.query('user_id in @double_users_list')
double_users_all_v2[['group','ab_test']].value_counts()

group  ab_test                
A      recommender_system_test    921
       interface_eu_test          819
B      interface_eu_test          783
       recommender_system_test    681
dtype: int64

Пользователи, которые попали в группу А "interface_eu_test" остаются в исследовании, поскольку это контрольная группа, и на нее не оказывается никакого влияния.

In [21]:
#пользователи, попавшие в группу В "interface_eu_test"
int_b = double_users_all_v2.query('ab_test=="interface_eu_test" & group =="B"')
users_int_b = int_b['user_id']

#выделим пользователей, попавших в группу В "interface_eu_test" и в группу А нашего теста
int_rec_a = double_users_all_v2.query('ab_test=="recommender_system_test" & user_id in @users_int_b & group =="A"')

#выделим пользователей, попавших в группу В "interface_eu_test" и в группу В нашего теста
int_rec_b = double_users_all_v2.query('ab_test=="recommender_system_test" & user_id in @users_int_b & group =="B"')

#всего пользователей нашего теста, попавших в группу А
rec_a = participants.query('ab_test=="recommender_system_test" & group =="A"')

#всего пользователей нашего теста, попавших в группу В
rec_b = participants.query('ab_test=="recommender_system_test" & group =="B"')

print('Процент задвоенных пользователей в группе А нашего теста:', len(int_rec_a)/len(rec_a)*100)
print('Процент задвоенных пользователей в группе В нашего теста::', len(int_rec_b)/len(rec_b)*100)

Процент задвоенных пользователей в группе А нашего теста: 11.480125523012552
Процент задвоенных пользователей в группе В нашего теста:: 11.956899548140425


Я посчитала, сколько из задвоенных пользователей попало в группу В конкурирующего теста (783). Затем нашла сколько из них пользователей попали в группу А и в группу В нашего теста (в А 439, в В 344). Затем нашла отношение задвоенных пользователй от общего числа пользователй групп А и В нашего теста. Получилось, что задвоенных пользовотелей примерно одинаковое количество: около 12%. Исходя из этого делаю вывод, что пользователеи распределены равномерно и их можно не удалять. Продолжаем тест дальше.

### 2.3. Проверка аудитории теста: в тест должно быть отобрано 15% новых пользователей из региона EU

In [22]:
#создадим отдельный датафрем test, в котром будут толлько участники recommender_system_test
test = participants.query('ab_test=="recommender_system_test"')

In [23]:
#ограничим период набора новых пользователей в соответствии с ТЗ
users = users.query('first_date<"2020-12-22"')

In [24]:
#создадим датафрем с пользователями только из EU согласно ТЗ
users_eu = users.query('region=="EU"')
print(users_eu.head(10))
print('Количество пользователей из Европы всего:', users_eu['user_id'].count())

             user_id first_date region   device
0   D72A72121175D8BE 2020-12-07     EU       PC
2   2E1BF1D4C37EA01F 2020-12-07     EU       PC
3   50734A22C0C63768 2020-12-07     EU   iPhone
7   8942E64218C9A1ED 2020-12-07     EU       PC
9   FFCEA1179C253104 2020-12-07     EU  Android
11  084A22B980BA8169 2020-12-07     EU  Android
12  8ACC2420471B31E4 2020-12-07     EU       PC
13  E6DE857AFBDC6102 2020-12-07     EU       PC
14  5BE017E9C8CC42F8 2020-12-07     EU  Android
15  7B6452F081F49504 2020-12-07     EU   iPhone
Количество пользователей из Европы всего: 42340


In [25]:
#создадим список пользователей только из Европы
users_eu_list = users_eu['user_id']

In [26]:
#найдем количество пользоватлелей из Европы в датафрейме test
print('Количество пользователей из Европы всего:', users_eu_list.nunique())
print('Количество пользователей из Европы из test:',len(test.query('user_id in @users_eu_list')))
print('Процент пользовталелей из Европы из test к общему количеству:', (len(test.query(
    'user_id in @users_eu_list')))/(users_eu_list.nunique())*100)

Количество пользователей из Европы всего: 42340
Количество пользователей из Европы из test: 6351
Процент пользовталелей из Европы из test к общему количеству: 15.0


Количество пользователей из Европы составляет ровно 15% от общего числа пользователей. Поскольку нас интерсуют только пользователи из Европы, оставим в датафрейме test только их

In [27]:
test = test.query('user_id in @users_eu_list')
print(test['user_id'].count())

6351


### 2.4. Проверим, есть ли пользователи, попавшие в обе группы исследуемого теста

In [28]:
#узнаем сколько участников в А и В
test['group'].value_counts()

A    3634
B    2717
Name: group, dtype: int64

In [29]:
double_users = test.groupby('user_id').agg({'group':'nunique'}).query('group>1')
users_nunique = test['user_id'].nunique()
print('Количество пользователей, одновременно попавших в обе группы:', len(double_users))
print('Количество уникальных пользователей, принявших участие в тесте:',users_nunique)
print('Процент пользователей, одновременно попавших в обе группы:', (len(double_users)/users_nunique)*100)

Количество пользователей, одновременно попавших в обе группы: 0
Количество уникальных пользователей, принявших участие в тесте: 6351
Процент пользователей, одновременно попавших в обе группы: 0.0


Пользователей, попавших в обе группы нет.

### 2.5. Сортировка данных за период: 14 дней с момента регистрации пользователи

In [30]:
#объединим таблици с пользователями и событиями
users_events = users.merge(events, on='user_id', how='left')

#выделим пользователей, которые участвуют в тесте после сортировок согласно ТЗ
users_test = test['user_id']

#оставим в датафрейме users_events пользователей, которые участвуют в тесте
users_events = users_events.query('user_id in @users_test')

In [31]:
#создадим столбец liftime
users_events['lifetime'] = (users_events['event_dt'] - users_events['first_date']).dt.days
users_events = users_events.reset_index(drop=True)
users_events.head(10)

<ipython-input-31-bfc6f6f4dd45>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_events['lifetime'] = (users_events['event_dt'] - users_events['first_date']).dt.days


,user_id,first_date,region,device,event_dt,event_name,details,lifetime
0,D72A72121175D8BE,2020-12-07,EU,PC,2020-12-07 21:52:10,product_page,NaN,0.0
1,D72A72121175D8BE,2020-12-07,EU,PC,2020-12-07 21:52:07,login,NaN,0.0
2,E6DE857AFBDC6102,2020-12-07,EU,PC,NaT,NaN,NaN,NaN
3,DD4352CDCF8C3D57,2020-12-07,EU,Android,2020-12-07 15:32:54,product_page,NaN,0.0
4,DD4352CDCF8C3D57,2020-12-07,EU,Android,2020-12-08 08:29:31,product_page,NaN,1.0
5,DD4352CDCF8C3D57,2020-12-07,EU,Android,2020-12-10 18:18:27,product_page,NaN,3.0
6,DD4352CDCF8C3D57,2020-12-07,EU,Android,2020-12-16 20:35:48,product_page,NaN,9.0
7,DD4352CDCF8C3D57,2020-12-07,EU,Android,2020-12-18 23:57:25,product_page,NaN,11.0
8,DD4352CDCF8C3D57,2020-12-07,EU,Android,2020-12-30 12:42:57,product_page,NaN,23.0
9,DD4352CDCF8C3D57,2020-12-07,EU,Android,2020-12-07 15:32:53,login,NaN,0.0


Проверим получившийся датафрем на пропуски

In [32]:
print(users_events.isna().sum());

user_id           0
first_date        0
region            0
device            0
event_dt       2870
event_name     2870
details       23094
lifetime       2870
dtype: int64


Выведем строки с пропусками и изучим их

In [33]:
users_events[users_events['event_name'].isna()].head()

,user_id,first_date,region,device,event_dt,event_name,details,lifetime
2,E6DE857AFBDC6102,2020-12-07,EU,PC,NaT,NaN,NaN,NaN
29,16C69ED14DA9F4A8,2020-12-07,EU,PC,NaT,NaN,NaN,NaN
33,2420EA89BB4BB0EA,2020-12-07,EU,iPhone,NaT,NaN,NaN,NaN
34,4D851A41ECF4440F,2020-12-07,EU,Android,NaT,NaN,NaN,NaN
74,8EB0C7622C73BBA3,2020-12-07,EU,Android,NaT,NaN,NaN,NaN


In [34]:
#согласно ТЗ отсортируем датафрем со значением ifetime меньше или равно 14
users_events = users_events.query('lifetime <= 14')
users_events.head(10)

,user_id,first_date,region,device,event_dt,event_name,details,lifetime
0,D72A72121175D8BE,2020-12-07,EU,PC,2020-12-07 21:52:10,product_page,NaN,0.0
1,D72A72121175D8BE,2020-12-07,EU,PC,2020-12-07 21:52:07,login,NaN,0.0
3,DD4352CDCF8C3D57,2020-12-07,EU,Android,2020-12-07 15:32:54,product_page,NaN,0.0
4,DD4352CDCF8C3D57,2020-12-07,EU,Android,2020-12-08 08:29:31,product_page,NaN,1.0
5,DD4352CDCF8C3D57,2020-12-07,EU,Android,2020-12-10 18:18:27,product_page,NaN,3.0
6,DD4352CDCF8C3D57,2020-12-07,EU,Android,2020-12-16 20:35:48,product_page,NaN,9.0
7,DD4352CDCF8C3D57,2020-12-07,EU,Android,2020-12-18 23:57:25,product_page,NaN,11.0
9,DD4352CDCF8C3D57,2020-12-07,EU,Android,2020-12-07 15:32:53,login,NaN,0.0
10,DD4352CDCF8C3D57,2020-12-07,EU,Android,2020-12-08 08:29:27,login,NaN,1.0
11,DD4352CDCF8C3D57,2020-12-07,EU,Android,2020-12-10 18:18:24,login,NaN,3.0


In [35]:
print(users_events.isna().sum()); # проверка на пропуски

user_id           0
first_date        0
region            0
device            0
event_dt          0
event_name        0
details       19705
lifetime          0
dtype: int64


Пропуски только по столбцу details, заполнение которого не являлось обязательным.

In [36]:
#создали итоговый датафрейм для работы
df = users_events.merge(test, on='user_id', how='left')
df.head(10)

,user_id,first_date,region,device,event_dt,event_name,details,lifetime,group,ab_test
0,D72A72121175D8BE,2020-12-07,EU,PC,2020-12-07 21:52:10,product_page,NaN,0.0,A,recommender_system_test
1,D72A72121175D8BE,2020-12-07,EU,PC,2020-12-07 21:52:07,login,NaN,0.0,A,recommender_system_test
2,DD4352CDCF8C3D57,2020-12-07,EU,Android,2020-12-07 15:32:54,product_page,NaN,0.0,B,recommender_system_test
3,DD4352CDCF8C3D57,2020-12-07,EU,Android,2020-12-08 08:29:31,product_page,NaN,1.0,B,recommender_system_test
4,DD4352CDCF8C3D57,2020-12-07,EU,Android,2020-12-10 18:18:27,product_page,NaN,3.0,B,recommender_system_test
5,DD4352CDCF8C3D57,2020-12-07,EU,Android,2020-12-16 20:35:48,product_page,NaN,9.0,B,recommender_system_test
6,DD4352CDCF8C3D57,2020-12-07,EU,Android,2020-12-18 23:57:25,product_page,NaN,11.0,B,recommender_system_test
7,DD4352CDCF8C3D57,2020-12-07,EU,Android,2020-12-07 15:32:53,login,NaN,0.0,B,recommender_system_test
8,DD4352CDCF8C3D57,2020-12-07,EU,Android,2020-12-08 08:29:27,login,NaN,1.0,B,recommender_system_test
9,DD4352CDCF8C3D57,2020-12-07,EU,Android,2020-12-10 18:18:24,login,NaN,3.0,B,recommender_system_test


In [37]:
print('Количество пользователей в итоговом датафреме:',df['user_id'].nunique())
print('Процент осташихся пользователей после фильтрации', df['user_id'].nunique()/users_nunique*100)

Количество пользователей в итоговом датафреме: 3481
Процент осташихся пользователей после фильтрации 54.8102660998268


Видим, что у 2870 пользователей отсутствовали события. В дальнейшем при фильтрации согласно ТЗ они удалились. И из 6351 пользоватлея осталось 3481, что составляет около 55% от датафрема до фильрации по лайфтайму. Это половина пользователей, однако для дальнейшего анализа эти пользователи не нужны.

## 3. Анализ

### 3.1. Посмотрим, какие события есть датафреме, как часто они встречаются. Отсортируем события по частоте.

In [38]:
events_count = df.groupby('event_name').agg({'user_id': 'count'}).sort_values('user_id', ascending=False)

#поменяем местами события в соответствии с логикой
events_count = events_count.reindex(labels=['login', 'product_page', 'product_cart', 'purchase'])
events_count

,user_id
event_name,
login,10313
product_page,6382
product_cart,3010
purchase,3123


Видов событий всего 4:

 - login (регистрация) - событие встречается чаще всего,
 - product_page(просмотр карточки товаров),
 - product_cart (просмотры корзины),
 - purchase (покупка).

Также видим, что покупку совершают чаще, чем просматривают товары в корзине. Можем предположить, что покупку можно совершить сразу на этапе просмотра карточкт товара без добавления его в корзину.

### 3.2. Посчитаем, сколько пользователей совершали каждое из этих событий. Отсортируем события по числу пользователей. Посчитаем долю пользователей, которые хоть раз совершали событие.

In [39]:
users_count = df.groupby('event_name').agg({'user_id': 'nunique'}).sort_values(by='user_id', ascending=False)

#поменяем местами события в соответствии с логикой
users_count = users_count.reindex(labels=['login', 'product_page', 'product_cart', 'purchase'])
users_count

,user_id
event_name,
login,3480
product_page,2178
product_cart,1026
purchase,1082


In [40]:
# Посчитаем долю пользователей, которые хоть раз совершали событие
users_count / len(df['user_id'].unique())*100

,user_id
event_name,
login,99.971273
product_page,62.568228
product_cart,29.474289
purchase,31.083022


Меньше всего пользователей перешли в корзину (29.5%). Почти все пользователи (99.97%) зарегистрировались. 62.57% пользоватлей перешли на страницу просмотра карточки товаров. 29.5% перешли в корзину и 31.08% оплатили покупку.

### 3.3. По воронке событий посчитаем, какая доля пользователей проходит на следующий шаг воронки (от числа пользователей на предыдущем).

In [41]:
users_count_df = df.groupby('event_name').agg({'user_id': 'nunique'}).sort_values(by='user_id', ascending=False)

#поменяем местами события в соответствии с логикой
users_count_df = users_count_df.reindex(labels=['login', 'product_page', 'product_cart', 'purchase'])
users_count_df['prev_step'] = users_count_df['user_id'].shift(periods=1, fill_value=0)
users_count_df['share'] = (users_count_df['user_id'] / users_count_df['prev_step'])*100

users_count_df

,user_id,prev_step,share
event_name,,,
login,3480,0,inf
product_page,2178,3480,62.586207
product_cart,1026,2178,47.107438
purchase,1082,1026,105.458090


Больше всего пользователей теряется на шаге перехода пользователей в корзину (около 53%).
Покупку совершает больше пользователей, чем перешло на страницу корзины. Это подтверждает, что купить товар можно не переходя на шаг добавления товара в корзину.


### Определим, сколько пользователей в каждой группе

In [42]:
df_user_counts = df.groupby('group').agg({'user_id': 'nunique'}).sort_values(by='user_id', ascending=False)

df_user_counts

,user_id
group,
A,2604
B,877


Пользователей группы В меньше чем в группе на более чем в три раза.

In [43]:
# Сформируем датафрем df_pivot_all, в котром отразим сколько раз было совершено каждое событие в каждой группе.
df_pivot_all = df.pivot_table(index='event_name', columns='group', values='user_id', aggfunc='count')

#поменяем местами события в соответствии с логикой
df_pivot_all = df_pivot_all.reindex(labels=['login', 'product_page', 'product_cart', 'purchase'])
df_pivot_all

group,A,B
event_name,,
login,7968,2345
product_page,5125,1257
product_cart,2385,625
purchase,2499,624


In [44]:
# Сформируем датафрем df_pivot_all_uniq, в котром отразим
#сколько раз было совершено каждое событие уникальными пользователями в каждой группе.

df_pivot_all_uniq = df.pivot_table(index='event_name', columns='group', values='user_id', aggfunc={'user_id': 'nunique'})

#поменяем местами события в соответствии с логикой
df_pivot_all_uniq = df_pivot_all_uniq.reindex(labels=['login', 'product_page', 'product_cart', 'purchase'])
print(df_pivot_all_uniq)

group            A    B
event_name             
login         2604  876
product_page  1685  493
product_cart   782  244
purchase       833  249


In [45]:
#создадим воронку для наглядного отражения количества уникальных пользователей на каждое событие
fig = go.Figure()
fig.update_layout(title={'text': "Воронка событий"})

fig.add_trace(go.Funnel(
    name = 'A',
    y = ["login", "product_page", "product_cart", "purchase"],
    x = [2604, 1685, 782, 833],
    textinfo = "value+percent initial"))


fig.add_trace(go.Funnel(
    name = 'B',
    orientation = "h",
    y = ["login", "product_page", "product_cart", "purchase"],
    x = [876, 493, 244, 249],
    textposition = "outside",
    textinfo = "value+percent total"))

fig.show()

 <div class="alert alert-success">
 <b>👍 Успех:</b> Отличная воронка!
 </div>

## 4. Изучение результатов эксперимента

### 4.1. Создадим функцию для проверки гипотез

In [46]:
#функция для проверки
def ztest(alpha, successes, trials):
    shidak = 1 - (1-alpha)**(1/4)

    # пропорция успехов в первой группе:
    p1 = successes[0]/trials[0]

    # пропорция успехов во второй группе:
    p2 = successes[1]/trials[1]

    # пропорция успехов в комбинированном датасете:
    p_combined = (successes[0] + successes[1]) / (trials[0] + trials[1])

    # разница пропорций в датасетах
    difference = p1 - p2

    # считаем статистику в ст.отклонениях стандартного нормального распределения
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/trials[0] + 1/trials[1]))

    # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = st.norm(0, 1)

    p_value = (1 - distr.cdf(abs(z_value))) * 2

    print('p-значение: ', p_value)

    if p_value < shidak:
        print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
    else:
        print('Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными')

### 4.2. Определим самое популярное событие (по количеству совершений)

In [47]:
most_popular_event = df['event_name'].value_counts().index[0]
print('Самое популярное событие:', most_popular_event)

Самое популярное событие: login


### 4.3. Посчитаем сколько раз было совершено это событи в контрольной группе

In [48]:
# посчитаем сколько раз было совершено событие в каждой из контрольных групп
df_pivot = df.pivot_table(index='event_name', columns='group', values='user_id', aggfunc='count'
                         ).query('event_name == @most_popular_event')
df_pivot

group,A,B
event_name,,
login,7968,2345


In [49]:
# Посчитаем долю этого события в каждой группе
shareA = df_pivot.loc['login', 'A'] / events_count.loc['login', 'user_id'] * 100
shareB = df_pivot.loc['login', 'B'] / events_count.loc['login', 'user_id'] * 100

print('Доля события login в группе А:', round(shareA, 2))
print('Доля события login в группе В:', round(shareB, 2))

Доля события login в группе А: 77.26
Доля события login в группе В: 22.74


### 4.4. Проверим, будет ли отличие между группами статистически достоверным по всем событиям между группами А и В.

```
H_0: Доли равны
H_a: Между долями есть разница
alpha = 0.05

Так как распределение статистики нормальное, вызовем метод cdf(). Саму статистику возьмём по модулю методом abs() —
чтобы получить правильный результат независимо от её знака. Это возможно, потому что тест двусторонний
```


#### 4.4.1. Проверяем нулевую гипотезу для события login

In [50]:
successes_A_B_login = np.array([df_pivot_all_uniq.loc['login', 'A'],
                               df_pivot_all_uniq.loc['login', 'B']])
trials_A_B_login = np.array([df_user_counts.loc['A', 'user_id'],df_user_counts.loc['B', 'user_id']])
ztest(0.05, successes_A_B_login, trials_A_B_login)

p-значение:  0.08481837035887363
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


Принимаем нулевую гипотезу, т.е. внедрение улучшенной рекомендательной системы не влияет на конверсию пользователей и на шаге регистрации (login) она равна в группах А и В.

#### 4.4.2. Проверяем нулевую гипотезу для события product_page

In [51]:
successes_A_B_product = np.array([df_pivot_all_uniq.loc['product_page', 'A'],
                               df_pivot_all_uniq.loc['product_page', 'B']])
trials_A_B_product = np.array([df_user_counts.loc['A', 'user_id'],df_user_counts.loc['B', 'user_id']])
ztest(0.05, successes_A_B_product, trials_A_B_product)

p-значение:  6.942739359416805e-06
Отвергаем нулевую гипотезу: между долями есть значимая разница


Отвергаем нулевую гипотезу. Т.е. внедрение улучшенной рекомендательной системы влияет на конверсию пользователей и на шаге просмотра товара (product_page) в группе А она больше чем в В. Или в контрольной группе А конверсия выше чем в экспериментальной В, и внесение изменений ухудшило конверсию на этом шаге.

#### 4.4.3. Проверяем нулевую гипотезу для события product_cart

In [52]:
successes_A_B_cart = np.array([df_pivot_all_uniq.loc['product_cart', 'A'],
                               df_pivot_all_uniq.loc['product_cart', 'B']])
trials_A_B_cart = np.array([df_user_counts.loc['A', 'user_id'],df_user_counts.loc['B', 'user_id']])
ztest(0.05, successes_A_B_cart, trials_A_B_cart)

p-значение:  0.21469192029582396
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


Принимаем нулевую гипотезу, т.е. внедрение улучшенной рекомендательной системы не влияет на конверсию пользователей и на шаге добавления в корзину (product_cart) она равна в группах А и В.

#### 4.4.4. Проверяем нулевую гипотезу для события purchase

In [53]:
successes_A_B_purchase = np.array([df_pivot_all_uniq.loc['purchase', 'A'],
                            df_pivot_all_uniq.loc['purchase', 'B']])
trials_A_B_purchase = np.array([df_user_counts.loc['A', 'user_id'],df_user_counts.loc['B', 'user_id']])
ztest(0.05, successes_A_B_purchase, trials_A_B_purchase)

p-значение:  0.04652482738393027
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


Принимаем нулевую гипотезу, т.е. внедрение улучшенной рекомендательной системы не влияет на конверсию пользователей и на шаге покупки (purchase) она равна в группах А и В.

Было проведено 4 сравнения, для большеймощности теста был применен метод Шидака. уровен значимости был расчитан по формуле shidak = 1 - (1-alpha)^(1/4), где alpha = 0.05

Нулевая гипотеза звучала: доли равны. Т.е. соотношения уникальных пользователей, совершивших то или иное событие, к общему числу уникальных пользователей приложения равны в группе А и группе В. Или другими словами внедрение улучшенной рекомендательной системы не влияет на конверсию пользователей и она равна в группах А и В. Чем меньше p-значение, тем больше определенность в том, что нулевая гипотеза является ложной, и что мы нашли подлинный эффект.

На шагах регистрация (login), добавление в корзину (product_cart) и покупка (purchase) нулевую гипотезу отвергаем.    
Это означает, что внедрение улучшенной рекомендательной системы не влияет на конверсию пользователей (т.е. не ухудшает и не улучшает ее). В связи с этим, внедрением улучшенной рекомендательной системы не принесло ожидаемых результатов.

Однако стоит отметить что на шаге просмотра товара (product_page) внедрение улучшенной рекомендательной системы оказало негативное влияние: конверсия на данном шаге в группе В снизилась, по сравнению с группой А.

Таким образом, признаем влиянеие внедрения улучшений негативным. Рекомендую не внедрять улучшенную рекомендательную систему.

## 5. Общий вывод по исследованию

Около 99,5% пользователей регистрируются в приложении.
На следующем шаге конверсия ухудшается: около 37% пользователей отваливаются на переходе к странице товара. Это довольно большое количество, поэто стоит обратить на это внимание и выявить причины (возможно, кнопка/ссылка для перехода к товару не слишком заметна, и пользователи просто ее не видят).
Товары в корзину добавляют лишь 47% от перешедшеих со страницы товара пользователей (с предыдщуего шага) или около 30% всех пользователей приложения. Это довольно низкие показатели. Стоит обратить внимание и на этот шаг.

Из добавивших товар в корзину, 100% пользователей успешно оплачивают его (около более 31% от всех пользователей). Также можем отметить, что пользователи могут сорешить покупку минуя шаг добавления товара в корзину.

Видим, что около 50% пользователей, перешедших на странцу с товаром, оплачивают товар. Это довольно неплохой показатель. Но его можно улучшить.

Рекомендую обратить внимание на переход на страницу товара и переход в корзину, выявить причины низкой конверсии и разработать предложения по ее увеличению.